In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

In [ ]:
train = pd.read_csv(r"../input/bike-sharing-demand/train.csv", parse_dates = ["datetime"])

In [ ]:
test = pd.read_csv(r"../input/bike-sharing-demand/test.csv", parse_dates = ["datetime"])

In [ ]:
train["year"] = train["datetime"].dt.year
train["hour"] = train["datetime"].dt.hour
train["dayofweek"] = train["datetime"].dt.dayofweek
train["month"] = train["datetime"].dt.month

test["year"] = test["datetime"].dt.year
test["hour"] = test["datetime"].dt.hour
test["dayofweek"] = test["datetime"].dt.dayofweek
test["month"] = test["datetime"].dt.month

In [ ]:
cols = list(train.columns.values)
cols.pop(cols.index('count'))
train = train[cols + ['count']]

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
_, axes = plt.subplots(1, 1, figsize = (20,12))
sns.boxplot(x = train["hour"], y = train["count"])

In [ ]:
fig, axes = plt.subplots(3,1, figsize = (20,12))

sns.countplot(train["season"], ax = axes[0], palette = "pastel")
sns.countplot(train["weather"], ax = axes[1], palette = "pastel")
sns.countplot(train["windspeed"], ax = axes[2])

In [ ]:
fig, axes = plt.subplots(3,1, figsize = (20,12))

sns.countplot(train["atemp"], ax = axes[0])
sns.countplot(train["temp"], ax = axes[1])
sns.countplot(train["month"], ax = axes[2])

In [ ]:
X = pd.DataFrame(train.iloc[:, :-1].values)
y = train.iloc[:, -1].values

In [ ]:
X

In [ ]:
X.drop(0, inplace = True, axis = 1)       #0 -> datetime
X.drop(6, inplace = True, axis = 1)       #6 -> atemp
X.drop(8, inplace = True, axis = 1)       #8 -> windspeed
X.drop(9, inplace = True, axis = 1)       #9 -> casual
X.drop(10, inplace = True, axis = 1)      #10 -> registered
X.drop(14, inplace = True, axis = 1)      #14 -> month
X = X.to_numpy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
# sc = StandardScaler()
# X_train[:, :] = sc.fit_transform(X_train[:, :])
# X_test[:, :] = sc.transform(X_test[:, :])

In [ ]:
# Building  Random Forest Regressor
regressor = RandomForestRegressor(n_estimators = 500, random_state = 0)
#regressor.fit(X_train, y_train)
y_train_log = np.log1p(y_train)
regressor.fit(X_train, y_train_log)

In [ ]:
# Evaluating on Training set
# reg_pred_train = regressor.predict(X_train)

In [ ]:
X_ans = pd.DataFrame(test.values)

In [ ]:
X_ans.drop(8, inplace = True, axis = 1)     #windspeed
X_ans.drop(6, inplace = True, axis = 1)     #atemp
X_ans.drop(0, inplace = True, axis = 1)     #datetime
X_ans.drop(12, inplace = True, axis = 1)    #month
X_ans = X_ans.to_numpy()

In [ ]:
reg_pred_test = regressor.predict(X_ans)

In [ ]:
reg_pred_test = pd.DataFrame(reg_pred_test)

In [ ]:
reg_pred_test.columns = ['count']

In [ ]:
answer = pd.concat((test.iloc[:, 0], np.exp(pd.DataFrame(reg_pred_test))), axis = 1)

In [ ]:
answer

In [ ]:
answer.to_csv('submission.csv', index = False)